<a href="https://colab.research.google.com/github/StrangeTcy/Book_NeuroevolutionThroughErlang/blob/master/improved_citywalls_processor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# #Add your email to the global config-file
# !git config --global user.email hatguy@yandex.ru
# # Add your username to the global config-file
# !git config --global user.name  StrangeTcy
# !git clone https://github.com/StrangeTcy/themachine
# !git add .
# !git commit -m "some improvements/progress in the citywalls processing notebook"
# !git push

In [ ]:
!pip install tqdm pywikibot wptools

     |████████████████████████████████| 512kB 5.7MB/s 
     |████████████████████████████████| 225kB 32.0MB/s 
  Created wheel for pywikibot: filename=pywikibot-5.1.0-cp36-none-any.whl size=536272 sha256=0786e90d3eefc7bf2c2b8e30b08e96f37db4c1d17a175560e80175ba0f47a6f0
  Stored in directory: /root/.cache/pip/wheels/38/fb/5d/947084957b2e1674bb89cf421da111435ca38178d35c1a75c2
  Created wheel for pycurl: filename=pycurl-7.43.0.6-cp36-cp36m-linux_x86_64.whl size=284379 sha256=e23828007113f3aed1b578cf80bae8c37c7a9cb6901d6ac1b7400e275dc16335
  Stored in directory: /root/.cache/pip/wheels/d9/a1/7b/2894883fd5c69a3ba684a49d255466d1e924b3cf58cedc1ade
Successfully built pywikibot pycurl


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# this was a one-time operation
# !cat /content/drive/My\ Drive/citywalls/improved_citywalls_part2.pkl /content/drive/My\ Drive/citywalls/improved_almost.pkl > /content/drive/My\ Drive/citywalls/improved_all.pkl

In [ ]:
!cp /content/drive/My\ Drive/citywalls/user-config.py /content/user-config.py

In [ ]:
import pickle
import json
import re
import pywikibot
import wptools
from collections import defaultdict

import requests # to get the page
from bs4 import BeautifulSoup # to parse the page

from tqdm import tqdm #pretty progress bars


# some global variables
ents_already_met = defaultdict(list)
for_initial = defaultdict(list)
first_sentence = defaultdict(list)

DEBUG = False


# if an integer is larger than
# some constant we don't know
#  neo4j runs into an "integer too large error"
neo4j_int_limit = pow(2,64)




def prettify(a, lower=True):
	aa = a.replace(" ","_").replace("'","").replace("&","and")\
		.replace("(","").replace(")","").replace("-","_").\
		replace(".","").replace(":","").replace(",","").\
		replace("/","").replace("\\","").replace("[","").replace("]","")
	if lower:
		return aa.lower()
	else:
		return aa	




def type_prettifier(arg, date=False):
	# input ("type_prettifier has received {} \n of type {}".format(arg, type(arg)))
	if isinstance(arg, str):
		if arg.isnumeric():
			if int(arg) < neo4j_int_limit:
				return int(arg)
			else:
				pretty_arg = "\""+arg+"\""
				return 	pretty_arg
		else:
			if date == False:
				try:
					float(arg)
					return float(arg)
				except ValueError:
					# we don't need extra double quotes
					# it fucks up cypher lines
					# if "\"" in arg:
					# 	# input ("Ok, there are double quotes in this arg:{}".format(arg))
					# 	arg = arg.replace('\"',"")
					# 	input ("And now we will return {}".format(arg))
					arg = arg.replace("\"","")
					pretty_arg = "\""+arg+"\""
					# input ("And now we will return {}".format(pretty_arg))
					return pretty_arg
			else:
				pretty_arg = "\""+str(arg)+"\""
				return 	pretty_arg
	if isinstance(arg, float) or isinstance(arg,int):
		if date == False:
			if arg < neo4j_int_limit:
				return arg
			else:
				pretty_arg = "\""+str(arg)+"\""
				return 	pretty_arg
		else:
			print ("arg is {} of type {} and date = {}".format(arg, type(arg), date))
			pretty_arg = "\""+str(arg)+"\""
			return 	pretty_arg
	if isinstance(arg, list):
		return arg
	# this might be a stupid way to store a dict
	# but it works
	if isinstance(arg, dict):
		# Yep, sometimes there are
		# double quotes in that dict
		arg_copy = defaultdict(list)
		for it in arg.items():
			new_it_0 = it[0].replace("\"","")
			new_it_1 = it[1].replace("\"","")
			arg_copy[new_it_0] = new_it_1
		pretty_arg = "\""+str(arg_copy)+"\""
		return 	pretty_arg





def meta_part_parser(meta):
	print ("meta is {}".format(meta))

	addresses = defaultdict(list)

	meta_set = set()
	for meta_it in meta:
		meta_it_list = meta_it.split(",")
		for _i_, meta_thingy in enumerate(meta_it_list):
			if "ул." in meta_thingy:
				print ("{}, {}".format(meta_thingy, meta_it_list[_i_+1]))
				addresses[meta_thingy] = meta_it_list[_i_+1]
			if "наб." in meta_thingy:
				print ("{}, {}".format(meta_thingy, meta_it_list[_i_+1]))
				addresses[meta_thingy] = meta_it_list[_i_+1]
			# "проспект" 
			if "пр." in meta_thingy:
				print ("{}, {}".format(meta_thingy, meta_it_list[_i_+1]))
				addresses[meta_thingy] = meta_it_list[_i_+1]	

			meta_set.add(meta_thingy)
		print ("\n{}".format(meta_it.split(",")))
	for m in meta_set:
		if "Фотографии" in m:
			possible_name = m.replace("Фотографии","")
			if DEBUG:	
				input ("Ok, perhaps the name of this thingy is {}".format(possible_name))
			for_initial['name'] = possible_name
	if DEBUG:	
		input ("meta_set is {}".format(meta_set))
	for_initial['addresses'] = addresses




def sav_match_parser(saved_matches):
	# print ("{}".format(saved_matches))
	for sav_match in saved_matches:
		print ("sav_match is {} of type {}".format(sav_match, type(sav_match)))
		sav_match_parts = re.findall(r"AddrPart\(value=(.*?), type='(.*?)'\)",sav_match)
		print ("sav_match_parts is {} of type {}".format(sav_match_parts, type(sav_match_parts)))
		if sav_match_parts != []:
			if sav_match_parts[0][1] == "улица":
				print("Found улица {}".format(sav_match_parts[0][0]))




def cypher_initial_former(var_name, name, Category, url, assoc_photos):
	# input ("We've just received assoc_photos {}\nof type {}".format(assoc_photos, type(assoc_photos)))
	if DEBUG:
		input ("At the moment our first_sentence dict is {}\nLet's use some data from here".format(first_sentence))

	if "Category" in for_initial:
		Category = prettify(for_initial['Category'], lower=False)
	else:
		Category = "House"
	input ("And our Category is {}".format(Category))	


	street_lines = []
	for _, addr in enumerate(for_initial['addresses'].items()):
		if "ул." in addr[0]:
			street_categ = "street"
		if "наб." in addr[0]:
			street_categ = "Embankment"
		if "пр." in addr[0]:
			street_categ = "Prospect"	
		# street_line = "MERGE (street_{}:{}) ON CREATE SET street_{}.name = {} ON MATCH SET street_{}.name = {}".format(_, street_categ, _, type_prettifier(addr[0]), _, type_prettifier(addr[0]) )
		property_part = "{"+"name:{}".format(type_prettifier(addr[0]))+"}"
		street_line = "MERGE (street_{}:{} {}) ".format(_, street_categ, property_part)
		street_lines.append(street_line)


	street_lines_str = '\n'.join(street_lines)	


	architects_lines = []
	if not for_initial['architects'] == []:
		for _,arch in enumerate(for_initial['architects']):
			property_part = "{"+"name:{}".format(type_prettifier(arch))+"}"
			architect_line = "MERGE (arch_{}:Person:Architect {}) ".format(_, property_part)
			architects_lines.append(architect_line)

	architects_lines_str = '\n'.join(architects_lines)	

	
	arch_style_name = "{"+"name:{}".format(type_prettifier(for_initial['arch_style']))+"}"
	cw_name_and_stuff =  "{"+"name:{}".format(type_prettifier(name))+", citywalls_url:{}".format(type_prettifier(url))+", assoc_photos:{}".format(type_prettifier(assoc_photos))+"}"
	

	cypher_initial = "MERGE (arch_style:Architectural_style {})\
					\nMERGE ({}:{} {} )\
					\nMERGE ({})-[:HAS_STYLE]->(arch_style) \
					".format(arch_style_name, 
						var_name, Category, 
						cw_name_and_stuff,
						var_name)
	
	addr_lines = []
	for _, addr in enumerate(for_initial['addresses'].items()):
		num_part = "{"+"number:{}".format(addr[1])+"}"
		addr_lines.append("MERGE ({})-[:IS_IN {}]->(street_{})".format(var_name, num_part, _))				
					
	addr_lines_str = '\n'.join(addr_lines)				

	
	built_lines = []
	for _,arch in enumerate(for_initial['architects']):
		constr_time = "{"+"time:{}".format(type_prettifier(for_initial['constr_year']))+"}"
		built_line = "MERGE (arch_{})-[:BUILT {}]->({}) ".format(_, constr_time  ,var_name)
		built_lines.append(built_line)
	built_lines_str = '\n'.join(built_lines)	


	cypher_initial = street_lines_str+"\n"+architects_lines_str+"\n"+cypher_initial+"\n"+addr_lines_str+"\n"+built_lines_str

	if DEBUG:	
		input ("We're about to return cypher_initial {}".format(cypher_initial))
	return cypher_initial				




def props_parser(props):
	print ("We've just got props {}".format(props))
	for prop in props.items():
		print ("{}: {}".format(prop[0], prop[1]))
		if "Архитекторы" in prop[0]:
			architects = re.findall(r"(\w*) (\w)\. (\w)\.", prop[1])
			
			print ("architects is now {} of type {}".format(architects, type(architects)))
			if not architects == []:
				print ("architects[0] = {} of type {}".format(architects[0], type(architects[0])))
				for a in architects[0]:
					print ("this tuple has element {}".format(a))
				architects_modified = ["{} {}. {}.".format(a[0],a[1],a[2]) for a in architects]
				print ("architects_modified is now {}".format(architects_modified))	
				for_initial['architects'] = architects_modified
				if DEBUG:
					input ("Architects...")


			# this is experimental stuf...	
			for arch in architects:		
				print ("arch is {}".format(arch))
				# site = pywikibot.Site("ru", "wikipedia")
				# # alright, we need an actual name for this
				# # not just initials
				# page = pywikibot.Page(site, "{}".format(arch))
				# item = pywikibot.ItemPage.fromPage(page)

				# print(item)
		elif "Год постройки" in prop[0]:
			constr_year = prop[1]
			for_initial['constr_year'] = constr_year
			print ("constr_year is {}".format(constr_year))
		elif "Стиль" in prop[0]:	
			arch_style = prop[1]
			for_initial['arch_style'] = arch_style
			print ("arch_style is {}".format(arch_style))	

	if DEBUG:	
		input ("And now our dict is {}".format(for_initial))			

	# return architects, constr_year, arch_style




def first_sentence_parser(sent):
	if DEBUG:
		input ("We've just received the first sentence: {}".format(sent))
	
	# let's look for familiar categories
	if "Mansion" in sent:
		if DEBUG:	
			input ("So, it's a mansion")
		for_initial['Category'] = "Mansion"
	if "Mansion" in sent and "outbuilding" in sent:
		if DEBUG:	
			input ("So, it's a mansion outbuilding")
		for_initial['Category'] = "Mansion outbuilding"	
	if "Palace" in sent:
		if DEBUG:	
			input ("So, it's a palace")
		for_initial['Category'] = "Palace"	
	if "Income House" in sent:
		if DEBUG:	
			input ("So, it's an Income House")
		for_initial['Category'] = "Income House"
	if "Dwelling House" in sent or "dwelling house" in sent or "Dwelling house" in sent:
		if DEBUG:	
			input ("So, it's an Dwelling House")
		for_initial['Category'] = "Dwelling House"	
	if "Boiler house" in sent:
		if DEBUG:	
			input ("So, it's an Boiler house")
		for_initial['Category'] = "Boiler house"	
	if "Profit House" in sent or "Profitable house" in sent or "profit house" in sent:
		if DEBUG:	
			input ("So, it's a Profit House")
		for_initial['Category'] = "Profit House"	
	if "Combination" in sent:
		if DEBUG:	
			input ("So, it's a Combination")
		for_initial['Category'] = "Combination"
	if "market" in sent:
		if DEBUG:	
			input ("So, it's a market")
		for_initial['Category'] = "market"
	if "Church" in sent:
		if DEBUG:	
			input ("So, it's a church")
		for_initial['Category'] = "Church"		
	if "Cathedral" in sent:
		if DEBUG:	
			input ("So, it's a Cathedral")
		for_initial['Category'] = "Cathedral"
	if "Convent" in sent:
		if DEBUG:	
			input ("So, it's a Convent")
		for_initial['Category'] = "Convent"		
	if "Club" in sent:
		if DEBUG:	
			input ("So, it's a Club")
		for_initial['Category'] = "Club"
	if "Library" in sent:
		if DEBUG:	
			input ("So, it's a Library")
		for_initial['Category'] = "Library"
	if "Nursery" in sent or "Nurseries" in sent:
		# if DEBUG:	
		# 	input ("So, it's a School")
		input ("So, it's a Nursery")
		for_initial['Category'] = "Nursery"	
	if "Production halls" in sent:
		# if DEBUG:	
		# 	input ("So, it's a School")
		input ("So, it's a Production halls")
		for_initial['Category'] = "Production halls"			
	if "Kindergarten" in sent:
		# if DEBUG:	
		# 	input ("So, it's a School")
		input ("So, it's a Kindergarten")
		for_initial['Category'] = "Kindergarten"
	if "School" in sent:
		# if DEBUG:	
		# 	input ("So, it's a School")
		input ("So, it's a School")
		for_initial['Category'] = "School"
	if "Gymnasium" in sent or "gymnasium" in sent:
		# if DEBUG:	
		# 	input ("So, it's a School")
		input ("So, it's a Gymnasium")
		for_initial['Category'] = "Gymnasium"	
	if "Boarding school" in sent:
		# if DEBUG:	
		# 	input ("So, it's a School")
		input ("So, it's a Boarding school")
		for_initial['Category'] = "Boarding school"
	if "Traffic school" in sent or "Traffic School" in sent:
		# if DEBUG:	
		# 	input ("So, it's a School")
		input ("So, it's a Traffic school")
		for_initial['Category'] = "Traffic school"
	if "Music school" in sent or "Music School" in sent:
		# if DEBUG:	
		# 	input ("So, it's a School")
		input ("So, it's a Music School")
		for_initial['Category'] = "Music School"			
	if "College" in sent:
		# if DEBUG:	
		# 	input ("So, it's a School")
		input ("So, it's a College")
		for_initial['Category'] = "College"	
	if "Institute" in sent:
		# if DEBUG:	
		# 	input ("So, it's a School")
		input ("So, it's a Institute")
		for_initial['Category'] = "Institute"
	if "University" in sent:
		# if DEBUG:	
		# 	input ("So, it's a School")
		input ("So, it's a University")
		for_initial['Category'] = "University"
	if "Academy" in sent:
		# if DEBUG:	
		# 	input ("So, it's a School")
		input ("So, it's a Academy")
		for_initial['Category'] = "Academy"		
	if "Bath" in sent or " bath" in sent:
		# if DEBUG:	
		# 	input ("So, it's a School")
		input ("So, it's a Bath")
		for_initial['Category'] = "Bath"
	if "Bank" in sent or " bank" in sent:
		# if DEBUG:	
		# 	input ("So, it's a School")
		input ("So, it's a Bank")
		for_initial['Category'] = "Bank"	
	if "public toilet" in sent:
		# if DEBUG:	
		# 	input ("So, it's a School")
		input ("So, it's a public toilet")
		for_initial['Category'] = "public toilet"			
	if "Factory" in sent:
		if DEBUG:	
			input ("So, it's a Factory")
		for_initial['Category'] = "Factory"
	if "Club" in sent:
		if DEBUG:	
			input ("So, it's a Club")
		for_initial['Category'] = "Club"	
	if "Orphanage" in sent:
		if DEBUG:	
			input ("So, it's an Orphanage")
		for_initial['Category'] = "Orphanage"
	if "Polyclinic" in sent or "polyclinic" in sent:
		if DEBUG:	
			input ("So, it's a Polyclinic")
		for_initial['Category'] = "Polyclinic"
	if "Hospital" in sent:
		if DEBUG:	
			input ("So, it's a Hospital")
		for_initial['Category'] = "Hospital"
	if "Maternity Hospital" in sent:
		if DEBUG:	
			input ("So, it's a Maternity Hospital")
		for_initial['Category'] = "Maternity Hospital"		
	if "Prophylaxis" in sent:
		if DEBUG:	
			input ("So, it's a Prophylaxis")
		for_initial['Category'] = "Prophylaxis"	
	if "Hospice" in sent:
		if DEBUG:	
			input ("So, it's a Hospice")
		for_initial['Category'] = "Hospice"	
	if "plant" in sent or "Plant" in sent:
		if DEBUG:	
			input ("So, it's a Plant")
		for_initial['Category'] = "Plant"
	if "dispensary" in sent:
		if DEBUG:	
			input ("So, it's a dispensary")
		for_initial['Category'] = "dispensary"	
	if "workshop" in sent:
		if DEBUG:	
			input ("So, it's a workshop")
		for_initial['Category'] = "workshop"
	if "Fire Protection Station" in sent:
		if DEBUG:	
			input ("So, it's a Fire Protection Station")
		for_initial['Category'] = "Fire Protection Station"		
	if "Residential Complex" in sent or "residential complex" in sent:
		if DEBUG:	
			input ("So, it's a Residential Complex")
		for_initial['Category'] = "Residential Complex"	
	if "Residential wing" in sent:
		if DEBUG:	
			input ("So, it's a Residential wing")
		for_initial['Category'] = "Residential wing"					
	if "residential building" in sent or "Residential house" in sent or "Residential building" in sent:
		if DEBUG:	
			input ("So, it's a residential building")
		for_initial['Category'] = "residential building"
	if "Non-residential building" in sent:
		if DEBUG:	
			input ("So, it's a Non-residential building")
		for_initial['Category'] = "Non-residential building"	
	if "Autoenterprise" in sent:
		if DEBUG:	
			input ("So, it's a Autoenterprise")
		for_initial['Category'] = "Autoenterprise"
	if "Fund" in sent:
		if DEBUG:	
			input ("So, it's a Fund")
		for_initial['Category'] = "Fund"
	if "metro station" in sent:
		if DEBUG:	
			input ("So, it's a metro station")
		for_initial['Category'] = "metro station"	
	if "Cemetery" in sent:
		if DEBUG:	
			input ("So, it's a Cemetery")
		for_initial['Category'] = "Cemetery"	
	if "stage" in sent:
		if DEBUG:	
			input ("So, it's a stage")
		for_initial['Category'] = "stage"	
	if "shopping complex" in sent:
		if DEBUG:	
			input ("So, it's a shopping complex")
		for_initial['Category'] = "shopping complex"	
	if "wood shed" in sent:
		if DEBUG:	
			input ("So, it's a wood shed")
		for_initial['Category'] = "wood shed"
	if "Wooden house" in sent:
		if DEBUG:	
			input ("So, it's a Wooden house")
		for_initial['Category'] = "Wooden house"	
	if "hostel" in sent or "Hostel" in sent:
		if DEBUG:	
			input ("So, it's a hostel")
		for_initial['Category'] = "hostel"
	if "Building" in sent:
		if DEBUG:	
			input ("So, it's a Building")
		for_initial['Category'] = "Building"
	if "Cinema" in sent or "cinema" in sent:
		if DEBUG:	
			input ("So, it's a Cinema")
		for_initial['Category'] = "Cinema"			
	if "Production building" in sent or "Production Building" in sent:
		if DEBUG:	
			input ("So, it's a Production building")
		for_initial['Category'] = "Production building"	
	if "Apartment building" in sent:
		if DEBUG:	
			input ("So, it's a Apartment building")
		for_initial['Category'] = "Apartment building"				
	if "Laboratory" in sent:
		if DEBUG:	
			input ("So, it's a Laboratory")
		for_initial['Category'] = "Laboratory"
	if "Museum" in sent:
		if DEBUG:	
			input ("So, it's a Museum")
		for_initial['Category'] = "Museum"
	if "Fire Depot" in sent:
		if DEBUG:	
			input ("So, it's a Fire Depot")
		for_initial['Category'] = "Fire Depot"	
	if "boiler room" in sent or "Boiler Room" in sent:
		if DEBUG:	
			input ("So, it's a boiler room")
		for_initial['Category'] = "boiler room"	
	if "Fair" in sent:
		if DEBUG:	
			input ("So, it's a Fair")
		for_initial['Category'] = "Fair"		
	if "Repair Base" in sent:
		if DEBUG:	
			input ("So, it's a Repair Base")
		for_initial['Category'] = "Repair Base"		
	if "Dormitory" in sent or "dormitory" in sent:
		if DEBUG:	
			input ("So, it's a Dormitory")
		for_initial['Category'] = "Dormitory"
	if "Hotel" in sent or "hotel" in sent:
		if DEBUG:	
			input ("So, it's a Hotel")
		for_initial['Category'] = "Hotel"		
	if "Center" in sent:
		if DEBUG:	
			input ("So, it's a Center")
		for_initial['Category'] = "Center"
	if "Department" in sent:
		if DEBUG:	
			input ("So, it's a Department")
		for_initial['Category'] = "Department"
	if "Police Department" in sent:
		if DEBUG:	
			input ("So, it's a Police Department")
		for_initial['Category'] = "Police Department"		
	if "Parking" in sent:
		if DEBUG:	
			input ("So, it's a Parking")
		for_initial['Category'] = "Parking"
	if "enterprise" in sent:
		if DEBUG:	
			input ("So, it's a enterprise")
		for_initial['Category'] = ""		
	if "Airport Terminal" in sent:
		if DEBUG:	
			input ("So, it's a Airport Terminal")
		for_initial['Category'] = "Airport Terminal"			
	if "Cash Centre" in sent:
		if DEBUG:	
			input ("So, it's a Cash Centre")
		for_initial['Category'] = "Cash Centre"	
	if "Veterinary station" in sent or "Veterinary Station" in sent:
		if DEBUG:	
			input ("So, it's a Veterinary station")
		for_initial['Category'] = "Veterinary station"
	if "Separate battalion" in sent:
		if DEBUG:	
			input ("So, it's a Separate battalion")
		for_initial['Category'] = "Separate battalion"			
	if "Business Center" in sent or "Business Centre" in sent or "Business center" in sent:
		if DEBUG:	
			input ("So, it's a Business Center")
		for_initial['Category'] = "Business Center"	
	if "Auto Salon" in sent:
		if DEBUG:	
			input ("So, it's a Auto Salon")
		for_initial['Category'] = "Auto Salon"
	if "Shop" in sent or " shop" in sent:
		if DEBUG:	
			input ("So, it's a Shop")
		for_initial['Category'] = "Shop"
	if "Office" in sent:
		if DEBUG:	
			input ("So, it's a Office")
		for_initial['Category'] = "Office"
	if "Youth Centre" in sent:
		if DEBUG:	
			input ("So, it's a Youth Centre")
		for_initial['Category'] = "Youth Centre"				
	if "Restaurant" in sent:
		if DEBUG:	
			input ("So, it's a Restaurant")
		for_initial['Category'] = "Restaurant"	
	if "Railway House" in sent:
		if DEBUG:	
			input ("So, it's a Railway House")
		for_initial['Category'] = "Railway House"
	if "railway station" in sent:
		if DEBUG:	
			input ("So, it's a railway station")
		for_initial['Category'] = "railway station"	
	if "warehouse" in sent:
		if DEBUG:	
			input ("So, it's a warehouse")
		for_initial['Category'] = "warehouse"		
	if "enterprise" in sent:
		if DEBUG:	
			input ("So, it's a enterprise")
		for_initial['Category'] = "enterprise"					
	if "Building of exemplary scales" in sent:
		if DEBUG:	
			input ("So, it's a Building of exemplary scales")
		for_initial['Category'] = "Building of exemplary scales"		
	if "Automobile service" in sent:
		if DEBUG:	
			input ("So, it's a Automobile service")
		for_initial['Category'] = "Automobile service"
	if "Tram Park" in sent:
		if DEBUG:	
			input ("So, it's a Tram Park")
		for_initial['Category'] = "Tram Park"
	if "Base" in sent:
		# if DEBUG:	
		# 	input ("So, it's a School")
		input ("So, it's a Base")
		for_initial['Category'] = "Base"
	if "mill" in sent:
		if DEBUG:	
			input ("So, it's a mill")
		for_initial['Category'] = "mill"
	if "Trust" in sent:
		if DEBUG:	
			input ("So, it's a Trust")
		for_initial['Category'] = "Trust"	
	if "courtyard wing" in sent or "courtyard outbuilding" in sent:
		if DEBUG:	
			input ("So, it's a courtyard wing")
		for_initial['Category'] = "courtyard wing"	
	if "Pakhaus" in sent:
		if DEBUG:	
			input ("So, it's a Pakhaus")
		for_initial['Category'] = "Pakhaus"
	if "Square" in sent:
		if DEBUG:	
			input ("So, it's a Square")
		for_initial['Category'] = "Square"	
	if "Khladokombinat" in sent or "Chladokombinat" in sent:
		if DEBUG:	
			input ("So, it's a Khladokombinat")
		for_initial['Category'] = "Khladokombinat"		
	if "distance of the way" in sent:
		if DEBUG:	
			input ("So, it's a distance of the way")
		for_initial['Category'] = "distance of the way"		
	if "CHPP" in sent:
		if DEBUG:	
			input ("So, it's a CHPP")
		for_initial['Category'] = "CHPP"
	if "Taxi Park" in sent:
		if DEBUG:	
			input ("So, it's a Taxi Park")
		for_initial['Category'] = "Taxi Park"
	if "Garage" in sent:
		if DEBUG:	
			input ("So, it's a Garage")
		for_initial['Category'] = "Garage"
	if "Laboratory" in sent:
		if DEBUG:	
			input ("So, it's a Laboratory")
		for_initial['Category'] = "Laboratory"
	if "Power station" in sent:
		if DEBUG:	
			input ("So, it's a Power station")
		for_initial['Category'] = "Power station"
	if "Station" in sent:
		if DEBUG:	
			input ("So, it's a Station")
		for_initial['Category'] = "Station"
	if "Manufacturing Building" in sent:
		if DEBUG:	
			input ("So, it's a Manufacturing Building")
		for_initial['Category'] = "Manufacturing Building"													
	if "Manufacturing" in sent:
		if DEBUG:	
			input ("So, it's a Manufacturing")
		for_initial['Category'] = "Manufacturing"					

	# if "House" in sent:
	# 	if DEBUG:	
	# 		input ("So, it's a House")
	# 	for_initial['Category'] = "House"				
	
	# else:
	# 	for_initial['Category'] = "House"				


	cool_regex_p1 = r"((\sarch\.(\s(.*?)(,\s)((arch\.\s)(.*?)\()?)?))"

	
	matches_p1 = re.finditer(cool_regex_p1, sent, re.MULTILINE)

	print ("==========BEGIN REGEX MATCHES 1==========")

	for matchNum, match in enumerate(matches_p1, start=1):
	    
	    print ("Match {matchNum} was found at {start}-{end}: {match}".format(matchNum = matchNum, start = match.start(), end = match.end(), match = match.group()))
	    
	    for groupNum in range(0, len(match.groups())):
	        groupNum = groupNum + 1
	        
	        print ("Group {groupNum} found at {start}-{end}: {group}".format(groupNum = groupNum, start = match.start(groupNum), end = match.end(groupNum), group = match.group(groupNum)))
	        gr = match.group(groupNum)
	        if  gr is not None:
	        	if not(gr == ", " or gr == "arch. " or gr.isnumeric() or "(" in gr):
		        	first_sentence['group_1_{}'.format(groupNum)] = gr

	print ("==========END REGEX MATCHES 1==========")        




	# these two regexes seem redundant
	# both are looking for architects


	# cool_regex_p2 = r"(((.*?)\s?)?(Architects: (.*?))\s(Year of construction:(\s*)((\d{4})(,\s))?((\d{4})-(\d{4}))))"
	cool_regex_p2 = r"(((.*?)\s?)?(Architects: (.*?))\s)"

	
	matches_p2 = re.finditer(cool_regex_p2, sent, re.MULTILINE)

	print ("==========BEGIN REGEX MATCHES 2==========")

	for matchNum, match in enumerate(matches_p2, start=1):
	    
	    print ("Match {matchNum} was found at {start}-{end}: {match}".format(matchNum = matchNum, start = match.start(), end = match.end(), match = match.group()))
	    
	    for groupNum in range(0, len(match.groups())):
	        groupNum = groupNum + 1
	        
	        print ("Group {groupNum} found at {start}-{end}: {group}".format(groupNum = groupNum, start = match.start(groupNum), end = match.end(groupNum), group = match.group(groupNum)))
	        gr = match.group(groupNum)
	        if  gr is not None:
	        	if not(gr == ", " or gr == "arch. " or gr.isnumeric() or "(" in gr):
		        	first_sentence['group_1_{}'.format(groupNum)] = gr

	print ("==========END REGEX MATCHES 1==========")

	# architects, constr_year, arch_style = props_parser(props) 

	# Now let's filter our dict
	# And hopefully get the actual architects' names
	# for it in first_sentence.items():
	# 	if it[1] == ", " or it[1] == "arch." or it[1].isnumeric():
	# 		del first_sentence[it[0]]



	print ("We already know (from props) that architest are {},\
	 constr_year is {} and Architectural_style is {}".
	 format(for_initial['architects'], 
	 	for_initial['constr_year'], 
	 	for_initial['arch_style']))


	print ("Now our first_sentence dict is {}".format(first_sentence))


	if DEBUG:	
		input ("Once you've inspected all the matches, please press any key")
	
	input ("Once you've inspected all the matches, please press any key")
	
	# return architects, constr_year, arch_style

	# return first_sentence





def get_statements(num):
	print ("We've just received num {}. Processing...".format(num))


	res = requests.get("https://www.wikidata.org/wiki/{}".format(num))
	#let's get the html page itself
	html_page = res.content

	# now let's parse the page
	soup = BeautifulSoup(html_page, 'html.parser')

	# first, let's only deal with the body of the page
	soup = soup.body

	#delete all the scripts
	scripts = soup.find_all('script')
	for script in scripts:
		script.decompose()	

	noindex = soup.find_all('noindex')
	for noi in noindex:
		noi.decompose()


	divs = soup.find_all("div", {'class':"wikibase-statementgroupview"})
	
	statements = defaultdict(list)

	for _,div in enumerate(divs):
		props = div.find_all("div",{'class':"wikibase-statementgroupview-property-label"})
		# print ("props = {} of type {}".format(props, type(props)))
		propert = props[0].find_all('a')[0].contents[0]#get('href') 
		# print ("property is {}".format(propert))
		ent_objs = div.find_all("div",{'class':"wikibase-snakview-value wikibase-snakview-variation-valuesnak"})
	
		if len(ent_objs) != 0:
			if ent_objs[0].find_all('a') == []:
				# if num == "Q35":
					
				# 	input ("ent_objs is {}".format(ent_objs))
				statements[propert] = ent_objs
				# print ("Empty list. Deal with it later")
			else:
				actual_ent_objs = ent_objs[0].find_all('a')
				# print ("actual_ent_objs = {}".format(actual_ent_objs))
				if isinstance(actual_ent_objs[0].contents[0], str):
					statements[propert] = actual_ent_objs[0].contents[0]
				else:
					statements[propert] = actual_ent_objs
		else:
			input ("Magic; ent_objs is {} \n of length {}".format(ent_objs, len(ent_objs)))
		# print ("++++++++++++++++++++++++++++++++++++++++++")


	# TODO: these sometimes run into disambiguation pages
	# choose the right meaning 


	print ("Magic of post-processing...")
	for it in statements.items():
		
		if it[0] == "image":
			img_url = it[1][0].get('href')
			statements[it[0]] = img_url
		if it[0] == "flag image":
			img_url = it[1][0].get('href')
			# input ("flag image is {}".format(img_url))
			statements[it[0]] = img_url
		if it[0] == "coordinate location":	
			lat = it[1][0].get('data-lat')
			longt = it[1][0].get('data-lon')
			statements[it[0]] = {"lat":lat, "longt":longt}
		if it[0] == "length":
			length_value = it[1][0].text
			statements[it[0]] = length_value 
		if it[0] == "Commons category":
			com_cat = it[1][0].text
			statements[it[0]] = com_cat 
		if it[0] == "winter view":
			w_v_url = it[1][0].get('href')
			statements[it[0]] =  w_v_url 
		if it[0] == "date of birth":
			d_o_b = it[1][0].text	
			statements[it[0]] = d_o_b
		if it[0] == "date of death":
			# for haha in it[1][0]:
			# 	print (haha)
			d_o_d = it[1][0].text[:-9]	
			# print ("d_o_d is {} of type {}".format(d_o_d, type(d_o_d)))
			statements[it[0]] = d_o_d


		if it[0] == "JSTOR topic ID":
			# print ("JSTOR topic id. Now, what to do about it...")
			jstor = it[1][0].text
			statements[it[0]] = jstor


		if it[0] == "transliteration":
			# print ("transliteration. Now, what to do about it...")
			translit = it[1][0].text
			statements[it[0]] = translit
			# input ("And now our transliteration is {}".format(translit))
		if it[0] == "native label":
			# print ("native label. Now, what to do about it...")
			nat_lab = it[1][0].text
			statements[it[0]] = nat_lab
			# input ("And now our native label is {}".format(nat_lab))


		if it[0] == "coat of arms image":
			print ("coat of arms image. Now, what to do about it...")
			c_o_a = it[1][0].get('href')
			statements[it[0]] = c_o_a	
		if it[0] == "signature":
			print ("signature. Now, what to do about it...")
			sign = it[1][0].get('href')
			statements[it[0]] = sign	


		# page banner  -- we probably don't need that
		if it[0] == "page banner" or it[0] == "pronunciation audio":
			# del statements[it[0]]  # we seemingly can't do it 
									# since it fucks up the items iterator
			statements[it[0]] = 0 #let's just replace it with nothing for now
		if it[0] == "start time":
			# for haha in it[1][0]:
			# 	print (haha)
			st_t = it[1][0].text[:-9]	
			# print ("d_o_d is {} of type {}".format(d_o_d, type(d_o_d)))
			statements[it[0]] = st_t	
		if it[0] == "end time":
			# for haha in it[1][0]:
			# 	print (haha)
			e_t = it[1][0].text[:-9]	
			# print ("d_o_d is {} of type {}".format(d_o_d, type(d_o_d)))
			statements[it[0]] = e_t	
		if it[0] == "number of deaths":
			# for haha in it[1][0]:
			# 	print (haha)
			# n_o_d = it[1][0].text[:-9]	
			n_o_d = it[1][0].text	
			# input ("And now the number of deaths is {}".format(n_o_d))
			# print ("d_o_d is {} of type {}".format(d_o_d, type(d_o_d)))
			statements[it[0]] = n_o_d
		if it[0] == "collage_image" or it[0] == "collage image":
			img_url = it[1][0].get('href')
			statements[it[0]] = img_url
		if  it[0] == "Commons gallery":
			gal = it[1][0].text
			# input ("gal is {}".format(gal))
			statements[it[0]] = gal		
		if  it[0] == "RA Collections ID":
			ra_id = it[1][0].text
			statements[it[0]] = ra_id


		if it[0] == "Soundex":
			sndx = 	it[1][0].text
			# input ("And now soundex is {}".format(sndx))
			statements[it[0]] = sndx

		if it[0] == "inception":
			incpt = 	it[1][0].text
			# input ("And now inception is {}".format(incpt))
			statements[it[0]] = incpt	

		if it[0] == "life expectancy":
			lexp = 	it[1]#[0]#.text
			ger_arr = list(range(0,80,4))
			lexp_dict = defaultdict(list)
			for _,l in enumerate(lexp):
				print ("num is {}, l is {}".format(_, l.text))
				if num == "Q159":
					if _%4 == 0:
						lexp_dict[lexp[_].text] = lexp[_+1].text
				if num == "Q183":
					if _ in ger_arr:
						lexp_dict[lexp[_].text] = lexp[_+1].text
			# input ("lexp_dict is now {}".format(lexp_dict))	
			lexp_line = "\"{"+', '.join(["{}:{}".format(l[0],l[1]) for l in lexp_dict.items()])+"}\""
			# input ("lexp_line is now {}".format(lexp_line))
			# input ("And now life expectancy is {}".format(lexp))
			 
			statements[it[0]] = lexp_line

		if it[0] == "official name":
			o_n = 	it[1][0].text
			# input ("And now official name is {}".format(o_n))
			statements[it[0]] = o_n	

		if it[0] == "Unicode character":
			unic = 	it[1][0].text
			# input ("And now Unicode character is {}".format(unic))
			statements[it[0]] = unic	

		# if it[0] == "named after":
		# 	naf = 	it[1][0].text
		# 	input ("And now named after is {}".format(naf))
		# 	statements[it[0]] = naf		
			
		if it[0] == "short name":
			sh_n = 	it[1]#[0]#.text
			sh_n_arr = []
			for _,sh in enumerate(sh_n):
				if _ == 3 or _==5:
					print ("element {} is {}".format(_,sh.text))
					sh_n_arr.append(sh.text)
			sh_n_line = "\"{"+', '.join(sh_n_arr)+"}\""
			# input ("And now short name is {}".format(sh_n_line))
			statements[it[0]] = sh_n_line	

		if it[0] == "demonym":
			dmnm = 	it[1]#[0].text
			dm_arr = []
			for _,d in enumerate(dmnm):
				print ("element {} is {}".format(_,d.text))
				dm_arr.append(d.text)
			dm_line = "\"{"+', '.join(dm_arr)+"}\""
			# input ("And now demonym is {}".format(dm_line))
			statements[it[0]] = dm_line

		if it[0] == "motto text":
			mt = it[1]#.text
			mt_arr = []
			for _,m in enumerate(mt):
				print ("{} m is {}".format(_, m.text))
				if _ == 0:
					mt_arr.append(m.text)
			mt_line = "\"{"+str(mt_arr[0])+"}\""
			# input ("motto text is {}".format(mt_line))
			statements[it[0]] = mt_line		

		if it[0] == "coordinates of easternmost point":
			cemp = it[1]
			lat = it[1][0].get('data-lat')
			longt = it[1][0].get('data-lon')
			cemp_line =  {"lat":lat, "longt":longt}
			statements[it[0]] = cemp_line
			# input ("coordinates of easternmost point is {}".format(cemp_line))
			statements[it[0]] = cemp_line

		if it[0] == "coordinates of northernmost point":
			cnmp = it[1]
			lat = it[1][0].get('data-lat')
			longt = it[1][0].get('data-lon')
			cnmp_line =  {"lat":lat, "longt":longt}
			statements[it[0]] = cnmp_line
			# input ("coordinates of northernmost point is {}".format(cnmp_line))
			statements[it[0]] = cnmp_line	

		if it[0] == "coordinates of southernmost point":
			csmp = it[1]
			lat = it[1][0].get('data-lat')
			longt = it[1][0].get('data-lon')
			csmp_line =  {"lat":lat, "longt":longt}
			statements[it[0]] = csmp_line
			# input ("coordinates of southernmost point is {}".format(csmp_line))
			statements[it[0]] = csmp_line		

		if it[0] == "coordinates of westernmost point":
			cwmp = it[1]
			lat = it[1][0].get('data-lat')
			longt = it[1][0].get('data-lon')
			cwmp_line =  {"lat":lat, "longt":longt}
			statements[it[0]] = cwmp_line
			# input ("coordinates of westernmost point is {}".format(cwmp_line))
			statements[it[0]] = cwmp_line		

		if it[0] == "head of government":
			hog = 	it[1]#[0].text
			# input ("And now head of government is {}".format(hog))
			statements[it[0]] = hog	

		if it[0] == "head of state":
			hos = 	it[1]#[0].text
			# input ("And now head of state is {}".format(hos))
			statements[it[0]] = hos

		if it[0] == "marriageable age":
			ma = it[1][0].text	
			# input ("marriageable age is {}".format(ma))
			statements[it[0]] = ma

		if it[0] == "age of majority":
			am = it[1][0].text	
			# input ("age of majority is {}".format(am))
			statements[it[0]] = am

		if it[0] == "age of consent":
			ac = it[1][0].text	
			# input ("age of consent is {}".format(ac))
			statements[it[0]] = ac		

		if it[0] == "population":
			pop = it[1]
			rus_arr = [0,3,6,9,14,19]
			hav_arr = [0]
			val_arr = [0]
			yarosl_arr = [2,5,9]
			tashkent_arr = [0]
			other_arr = [0,3,6,9,14]
			czech_arr = [0,3,5]
			arctic_arr = [0]
			yugosl_arr = czech_arr
			petrz_arr = [0,2,5,9]
			suzdal_arr = [0,2,9]
			odessa_arr = [0]
			krak_arr = [0,4]
			sofia_arr = [0]
			kaling_arr = [0]
			antarc_arr = [0,5]
			eskimo_arr = [0]
			
			 
			pop_dict = defaultdict(list)
			for _,p in enumerate(pop):
				print ("{} is {}".format(_,p.text))
				if num == "Q131242":
					if _ in antarc_arr:
						pop_dict[pop[_].text] = pop[_+2].text
				elif num == "Q51":
					if _ in antarc_arr:
						pop_dict[pop[_].text] = pop[_+2].text
				elif num == "Q1829":
					if _ in kaling_arr:
						pop_dict[pop[_].text] = pop[_+1].text
				elif num == "Q472":
					if _ in sofia_arr:
						pop_dict[pop[_].text] = pop[_+1].text
				elif num == "Q18628307":
					if _ in krak_arr:
						pop_dict[pop[_].text] = pop[_+1].text
				elif num == "Q1874":
					if _ in odessa_arr:
						pop_dict[pop[_].text] = pop[_+1].text
				elif num == "Q15757":
					if _ in suzdal_arr:
						pop_dict[pop[_].text] = pop[_+1].text
				elif num == "Q1895":
					if _ in petrz_arr:
						pop_dict[pop[_].text] = pop[_+1].text
				elif num == "Q159":
					if _ in rus_arr:
						pop_dict[pop[_].text] = pop[_+1].text
				elif num == "Q1563":
					if _ in hav_arr:
						pop_dict[pop[_].text] = pop[_+1].text
				elif num == "Q36704":
					if _ in yugosl_arr:
						pop_dict[pop[_].text] = pop[_+1].text		
				elif num == "Q220622":
					if _ in val_arr:
						pop_dict[pop[_].text] = pop[2].text
				elif num == "Q25322":
					if _ in arctic_arr:
						pop_dict[pop[_].text] = pop[3].text		
				elif num == "Q269":
					if _ in tashkent_arr:
						pop_dict[pop[_].text] = pop[1].text
				elif num == "Q2423":
					if _ in yarosl_arr:
						pop_dict[pop[_].text] = pop[_+1].text
				elif num == "Q33946":
					if _ in czech_arr:
						pop_dict[pop[_].text] = pop[_+1].text									
				else:
					if _ in other_arr:
						pop_dict[pop[_].text] = pop[_+1].text
			pop_line = 	"\"{"+', '.join(["{}:{}".format(l[0],l[1]) for l in pop_dict.items()])+"}\""	
			# input ("and now population is {}".format(pop_line))	
			statements[it[0]] = pop_line

		if it[0] == "BTI Governance Index":
			bti = it[1]
			bti_dict = defaultdict(list)
			for _,b in enumerate(bti):
				print ("{} is {}".format(_,b.text))
				if _%3 == 0:
					bti_dict[bti[_].text] = bti[_+1].text
			bti_line = "\"{"+', '.join(["{}:{}".format(l[0],l[1]) for l in bti_dict.items()])+"}\""		
			# input ("bti governance is {}".format(bti_line))
			statements[it[0]] = bti_line

		if it[0] == "BTI Status Index":
			bti = it[1]
			eston_bti = [0, 3, 7, 10, 13, 16, 19]
			roman_bti = [0, 3, 6, 9, 12, 15, 19]
			moldova_bti = [0,3,6,9,12]
			
			bti_dict = defaultdict(list)
			for _,b in enumerate(bti):
				print ("{} is {}".format(_,b.text))
				if num == "Q217":
					if _ in moldova_bti:
						bti_dict[bti[_].text] = bti[_+1].text
				elif num == "Q191":
					# input ("Dealing with Estonian BTI")
					if _ in eston_bti:
						bti_dict[bti[_].text] = bti[_+1].text
				elif num == "Q218":
					if _ in roman_bti:
						bti_dict[bti[_].text] = bti[_+1].text		
				else:
					# input ("num is {}, dealing with BTI".format(num))
					if _%3 == 0:
						bti_dict[bti[_].text] = bti[_+1].text
			bti_line = "\"{"+', '.join(["{}:{}".format(l[0],l[1]) for l in bti_dict.items()])+"}\""		
			# input ("bti is {}".format(bti_line))
			statements[it[0]] = bti_line

		if it[0] == "compulsory education (minimum age)":
			cema = 	it[1][0].text
			# input ("And now compulsory education (minimum age) is {}".format(cema))
			statements[it[0]] = cema	

		if it[0] == "compulsory education (maximum age)":
			if num == "Q183":
				 cema = 	it[1]#[0].text
				#  input ("And now compulsory education (maximum age) is {}".format(cema))
			else:
				cema = 	it[1][0].text
				# input ("And now compulsory education (maximum age) is {}".format(cema))
			statements[it[0]] = cema

		if it[0] == "maximum temperature record":
			mtr = 	it[1][0].text
			# input ("And now maximum temperature record is {}".format(mtr))
			statements[it[0]] = mtr				

		if it[0] == "Human Development Index":
			hdi = 	it[1]#[0].text
			hdi_dict = defaultdict(list)
			sub_arr = list(range(25,141,4))
			# input("here're our indices: {}".format(sub_arr))
			for _,h in enumerate(hdi):
				print ("{} is {}".format(_,h.text))
				
				# if _ in [0,3,5,8,10,13,16,19,22,25,29,33,37,41,45,49,53,57,61,65,69,73,77,81,85,89,93,97]:
				if _ in [0,3,5,8,10,13,16,19,22]+sub_arr:
					hdi_dict[hdi[_].text] = hdi[_+1].text
			hdi_line = 	"\"{"+', '.join(["{}:{}".format(l[0],l[1]) for l in hdi_dict.items()])+"}\""	
			# input ("And now Human Development Index is {}".format(hdi_line))
			statements[it[0]] = hdi_line

		if it[0] == "inflation rate":
			ir = 	it[1]#[0].text
			ir_dict = defaultdict(list)
			# sub_arr = list(range(25,141,4))
			# input("here're our indices: {}".format(sub_arr))
			for _,h in enumerate(ir):
				print ("{} is {}".format(_,h.text))
				if _%3 == 0:
					print ("now at {}".format(_))
					ir_dict[ir[_].text] = ir[_+1].text
			ir_line = 	"\"{"+', '.join(["{}:{}".format(l[0],l[1]) for l in ir_dict.items()])+"}\""	
			# input ("And now inflation rate is {}".format(ir_line))
			statements[it[0]] = ir_line	

		if it[0] == "shares border with":
			bw = 	it[1]#[0].text	
			# for _,b in enumerate(bw):
			# 	print ("{} is {}".format(_,b))
			# input ("Hey ho, bordering countries")
			statements[it[0]] = bw

		if it[0] == "speed limit":
			if num == "Q183":
				sl = 	it[1]
				# input ("German speed limit is {}".format(sl))
				sl_line = sl	
			else:
				sl = 	it[1]#[0].text	
				sl_dict = defaultdict(list)
				for _,b in enumerate(sl):
					print ("{} is {}".format(_,b.text))
					if _%2 ==0:
						sl_dict[sl[_].text] = sl[_+1].text
				sl_line = 	"\"{"+', '.join(["{}:{}".format(l[0],l[1]) for l in sl_dict.items()])+"}\""	
				# input ("Hey ho, speed limit:{}".format(sl_line))
			statements[it[0]] = sl_line	

		if it[0] == "mains voltage":
			mv = 	it[1]#[0].text	
			mv_dict = defaultdict(list)
			for _,b in enumerate(mv):
				print ("{} is {}".format(_,b.text))
				if _ ==0:
					mv_dict[mv[_].text] = mv[_+1].text
			mv_line = 	"\"{"+', '.join(["{}:{}".format(l[0],l[1]) for l in mv_dict.items()])+"}\""	
			# input ("Hey ho, mains voltage:{}".format(mv_line))
			statements[it[0]] = mv_line	

		if it[0] == "IPA transcription":
			ipa = 	it[1]#[0].text	
			pk_arr = [0,3,7]
			amst_arr = [0,3,5,7,9,11]
			ipa_dict = defaultdict(list)
			for _,b in enumerate(ipa):
				print ("{} is {}".format(_,b.text))
				if num == "Q7951":
					if _ in pk_arr:
						ipa_dict[ipa[_].text] = ipa[_+1].text
				elif num == "Q727":
					if _ in amst_arr:
						ipa_dict[ipa[_].text] = ipa[_+1].text
				else:
					if _%2 ==0:
						ipa_dict[ipa[_].text] = ipa[_+1].text
			ipa_line = 	"\"{"+', '.join(["{}:{}".format(l[0],l[1]) for l in ipa_dict.items()])+"}\""	
			# input ("Hey ho, IPA transcription:{}".format(ipa_line))
			statements[it[0]] = ipa_line



		if it[0] == "spoken text audio":
			# input ("spoken audio found")
			statements[it[0]] = 0 #ignore that shit

		if it[0] == "pronunciation audio":
			# input ("pronunciation audio")# is {}".format(it[1]))
			statements[it[0]] = 0 #ignore that shit

		if it[0] == "top-level Internet domain":
			tld = it[1]#[0].text
			# input ("tld is {}".format(tld))
			statements[it[0]] = tld

		if it[0] == "VAT-rate":
			vat = it[1][0].text
			# input ("VAT is {}".format(vat))
			statements[it[0]] = vat

		if it[0] == "area":
			# input ("Yoohoo, num is {}".format(num))
			ar_line =  "\"{"+" "+"}\""
			stuff_arr = ["Q7525", "Q34266", "Q90", "Q183", "Q159", "Q649", "Q656", "Q727","Q803", "Q908", "Q1055", "Q1874", "Q2144",
			             "Q5332", "Q14657", "Q102158", "Q156732",  "Q4361264"]
			re_arr = amst_arr = par_arr = germ_arr = ru_arr = mosc_arr = sar_arr = [0]
			area = it[1]
			area_dict = defaultdict(list)
		# if _ == 0:
			if num == "Q1754":
				ar_line = "\"{"+area+"}\""
			elif num in stuff_arr:
				if _ in ru_arr:
					area_dict[area[_].text] = area[_+1].text
					ar_line = "\"{"+', '.join(["{}:{}".format(l[0],l[1]) for l in area_dict.items()])+"}\""
			else:
				for _,a in enumerate(area):
					print ("{} is {}".format(_,a.text))
					area_dict[area[_].text] = area[_+1].text
					ar_line = "\"{"+', '.join(["{}:{}".format(l[0],l[1]) for l in area_dict.items()])+"}\""
	
			# ar_line = area[0].text	
			# input ("area is {}".format(ar_line))
			statements[it[0]] = ar_line

		if it[0] == "locator map image":
			lmi = it[1][0].get('href')
			# input("locator map image is {}".format(lmi))
			statements[it[0]] = lmi

		if it[0] == "official website":
			ow = it[1]
			# input ("official website is {}".format(ow))
			statements[it[0]] = ow

		if it[0] == "mobile country code":
			mcc = it[1][0].text
			# input ("mobile country code is {}".format(mcc))
			statements[it[0]] = mcc

		if it[0] == "trunk prefix":
			tp = it[1][0].text
			# input ("trunk prefix is {}".format(tp))
			statements[it[0]] = tp

		if it[0] == "country calling code":
			ccc = it[1][0].text
			# for _,c in enumerate(ccc):
			# 	print ("{} is {}".format(_,c.text))
			# input ("country calling code is {}".format(ccc))
			statements[it[0]] = ccc

		if it[0] == "total fertility rate":
			tfr = 	it[1]#[0].text	
			tfr_dict = defaultdict(list)
			for _,b in enumerate(tfr):
				print ("{} is {}".format(_,b.text))
				if _%5 ==0:
					tfr_dict[tfr[_].text] = tfr[_+1].text
			tfr_line = 	"\"{"+', '.join(["{}:{}".format(l[0],l[1]) for l in tfr_dict.items()])+"}\""	
			# input ("Hey ho, total fertility rate:{}".format(tfr_line))
			statements[it[0]] = tfr_line	

		if it[0] == "nominal GDP":
			ngdp = 	it[1]#[0].text	
			ngdp_dict = defaultdict(list)
			for _,b in enumerate(ngdp):
				print ("{} is {}".format(_,b.text))
				if _%6 ==0:
					ngdp_dict[ngdp[_].text] = ngdp[_+1].text
			ngdp_line = 	"\"{"+', '.join(["{}:{}".format(l[0],l[1]) for l in ngdp_dict.items()])+"}\""	
			# input ("Hey ho, nominal GDP:{}".format(ngdp_line))
			statements[it[0]] = ngdp_line	

		if it[0] == "nominal GDP per capita":
			ngdp = 	it[1]#[0].text	
			ngdp_dict = defaultdict(list)
			sub_arr = list(range(8,153,5))
			for _,b in enumerate(ngdp):
				print ("{} is {}".format(_,b.text))
				if _ in [0,3,6]+sub_arr:
					ngdp_dict[ngdp[_].text] = ngdp[_+1].text
			ngdp_line = 	"\"{"+', '.join(["{}:{}".format(l[0],l[1]) for l in ngdp_dict.items()])+"}\""	
			# input ("Hey ho, nominal GDP per capita:{}".format(ngdp_line))
			statements[it[0]] = ngdp_line	

		if it[0] == "GDP (PPP)":
			ngdp = 	it[1]#[0].text	
			ngdp_dict = defaultdict(list)
			# sub_arr = list(range(8,153,5))
			for _,b in enumerate(ngdp):
				print ("{} is {}".format(_,b.text))
				if _%5 == 0:
					ngdp_dict[ngdp[_].text] = ngdp[_+1].text
			ngdp_line = 	"\"{"+', '.join(["{}:{}".format(l[0],l[1]) for l in ngdp_dict.items()])+"}\""	
			# input ("Hey ho, GDP (PPP):{}".format(ngdp_line))
			statements[it[0]] = ngdp_line	

		if it[0] == "PPP GDP per capita":
			ngdp = 	it[1]#[0].text	
			ngdp_dict = defaultdict(list)
			# sub_arr = list(range(8,153,5))
			for _,b in enumerate(ngdp):
				print ("{} is {}".format(_,b.text))
				if _%5 == 0:
					ngdp_dict[ngdp[_].text] = ngdp[_+1].text
			ngdp_line = 	"\"{"+', '.join(["{}:{}".format(l[0],l[1]) for l in ngdp_dict.items()])+"}\""	
			# input ("Hey ho, PPP GDP per capita:{}".format(ngdp_line))
			statements[it[0]] = ngdp_line


		if it[0] == "total exports":
			te = it[1]#[0].get('href')
			te_dict = defaultdict(list)
			for _,t in enumerate(te):
				print ("{} is {}".format(_,t.text))
				if _==0:
					te_dict[te[_].text] = te[_+1].text
			te_line = "\"{"+', '.join(["{}:{}".format(l[0],l[1]) for l in te_dict.items()])+"}\""		
			# input("total exports is {}".format(te_line))
			statements[it[0]] = te_line

		if it[0] == "total reserves":
			tr = it[1]#[0].get('href')
			polish_arr = [0, 6, 11,16]
			tr_dict = defaultdict(list)
			for _,t in enumerate(tr):
				print ("{} is {}".format(_,t.text))
				if num == "Q36":
					if _ in polish_arr:
						tr_dict[tr[_].text] = tr[_+1].text
				else: 
					if _%5 ==0:
						tr_dict[tr[_].text] = tr[_+1].text
			tr_line = "\"{"+', '.join(["{}:{}".format(l[0],l[1]) for l in tr_dict.items()])+"}\""		
			# input("total reserves is {}".format(tr_line))
			statements[it[0]] = tr_line

		if it[0] == "real gross domestic product growth rate":
			rgdpgr = it[1][0].text
			# input ("real gross domestic product growth rate is {}".format(rgdpgr))
			statements[it[0]] = rgdpgr

		if it[0] == "Gregorian calendar start date":
			gcsd = it[1][0].text.replace("Gregorian","")
			# input ("Gregorian calendar start date is {}".format(gcsd))
			statements[it[0]] = gcsd

		if it[0] == "GOST 7.67 cyrillic":
			gost = it[1][0].text#.replace("Gregorian","")
			# input ("GOST 7.67 cyrillic is {}".format(gost))
			statements[it[0]] = gost

		if it[0] == "ITU letter code":
			ilc = it[1][0].text
			# input ("ITU letter code is {}".format(ilc))
			statements[it[0]] = ilc

		if it[0] == "IAB code":
			iab = it[1][0].text
			# input ("IAB code is {}".format(iab))
			statements[it[0]] = iab

		if it[0] == "licence plate code":
			iab = it[1][0].text
			# input ("licence plate code is {}".format(iab))
			statements[it[0]] = iab

		if it[0] == "UIC alphabetical country code":
			iab = it[1][0].text
			# input ("UIC alphabetical country code is {}".format(iab))
			statements[it[0]] = iab

		if it[0] == "UIC numerical country code":
			iab = it[1][0].text
			# input ("UIC numerical country code is {}".format(iab))
			statements[it[0]] = iab		

		if it[0] == "maritime identification digits":
			iab = it[1][0].text
			# input ("maritime identification digits is {}".format(iab))
			statements[it[0]] = iab

		if it[0] == "INSEE countries and foreign territories code":
			iab = it[1][0].text
			# input ("INSEE countries and foreign territories code is {}".format(iab))
			statements[it[0]] = iab

		if it[0] == "IOC country code":
			iab = it[1][0].text
			# input ("IOC country code is {}".format(iab))
			statements[it[0]] = iab

		if it[0] == "WIPO ST.3":
			iab = it[1][0].text
			# input ("WIPO ST.3 is {}".format(iab))
			statements[it[0]] = iab	

		if it[0] == "GS1 country code":
			iab = it[1][0].text
			# input ("GS1 country code is {}".format(iab))
			statements[it[0]] = iab

		if it[0] == "ISO 3166-1 alpha-3 code":
			iab = it[1][0].text
			# input ("ISO 3166-1 alpha-3 code is {}".format(iab))
			statements[it[0]] = iab

		if it[0] == "ISO 3166-1 numeric code":
			iab = it[1][0].text
			# input ("ISO 3166-1 numeric code is {}".format(iab))
			statements[it[0]] = iab

		if it[0] == "Wolfram Language entity code":
			iab = it[1]#[0].text
			# input ("Wolfram Language entity code is {}".format(iab))
			statements[it[0]] = iab

		if it[0] == "local dialing code":
			ldc = it[1][0].text
			# input ("local dialing code = {}".format(ldc))
			statements[it[0]] = ldc

		if it[0] == "elevation above sea level":
			ldc = it[1][0].text
			# input ("elevation above sea level = {}".format(ldc))
			statements[it[0]] = ldc	 

	for it in statements.items():
		print ("{} : {}".format(it[0], it[1]))	

	return statements	





def ent_cypher_line_generator(ent, num, un_num, ent_num, categs, confidence, item_stuff=None, item_claims=None):
		
	# print ("ent_cypher_line_generator has just received item_stuff of {}".format(item_stuff))	

	# input ("len(ent[0]) is {}".format(len(ent[0])))
	if len(ent[0]) == 1: 
		# input("{} has length {}. That's damn suspicious!..".format(ent[0], len(ent[0])))
		

		# it's unlikely we'll ever need to deal
	    # with one-letter concepts
	    # so let's ignore them
		return " " 


	if ent[0] in ents_already_met:
		print ("We've dealt with this stuff already")

		
		# !!!!!!!! TODO: investigate !!!!!!!!!!!!!!!!!
		# POTENTIAL TROUBLE SOURCE!! 
		# seems to work alright
		
		# return (ents_already_met[ent[0]])
		return None	
	else:
		print ("we have {} ents already".format(len(ents_already_met)))
		print ("Oh, this is new!")
		# do all the processing and then ...


	# stuff_lines = []	
	stuff_str = ""

	
	if "Wikimedia disambiguation page" in categs or\
		"Wikimedia human name disambiguation page" in categs:
		# input ("I notice i am confused...")
		categs = categs.replace(":Wikimedia disambiguation page", "") # this shouldn't be used as a category
		categs = categs.replace(":Wikimedia human name disambiguation page", "")
		if DEBUG:
			input ("And now categs is {}".format(categs))
		
		stuff_str += ",REQUIRES_FURTHER_RESEARCH:\"True\""



	claim_str = ""

	if item_claims is not None:
		print ("item_claims keys are {}".format(item_claims.keys()))

		
		if item_stuff is not None:
			print ("item_stuff keys are {}".format(item_stuff.keys()))
		
			
			for stuff in item_stuff.items():
				if (stuff[0] == "item_desc_ru" and stuff[1] == "страница значений")\
				or \
				(stuff[0] == "item_desc_en" and stuff[1] == "Wikimedia disambiguation page"):
					# input("Oh dear, we're in trouble NOW!")
					
					stuff_str += ",REQUIRES_FURTHER_RESEARCH:\"True\""
				# stuff_line = "entity_{}_{}_{}.{} = {}".format(num, un_num, ent_num, stuff[0], type_prettifier(stuff[1]) )
				stuff_str += ",{}:{}".format(stuff[0], type_prettifier(stuff[1]))
				

		# input ("Now stuff_str is {}".format(stuff_str))	


		# claim_lines = []
		

		for claim in item_claims.items():
			# print ("{} : {}".format(claim[0], claim[1]))


			if claim[0] == "coordinate location":
				print ("Got coordinates here!!")
				claim_str += ",lat:{}, longt:{}".format(type_prettifier(claim[1]['lat']), type_prettifier(claim[1]['longt']))
			else:
				claim_str += ",{}:{}".format(prettify(claim[0]), type_prettifier(claim[1]))
			
		

		if DEBUG:
			input ("Now claim_str is {}".format(claim_str))	

		cat_name = "categorization_confidence:{}, name:{}".format(type_prettifier(confidence), type_prettifier(ent[0]))
		
		if DEBUG:	
			input ("Once you're satisfied with all the claim lines, press any key..")	
		
				

	else:
		cat_name = "categorization_confidence:{}, name:{}".format(type_prettifier(confidence), type_prettifier(ent[0]))


	relations_part = "MERGE (sent_{}_{})-[:MENTIONS]->(entity_{}_{}_{})".format(num, un_num, num, un_num, ent_num)


	all_ent_props = "{"+cat_name+stuff_str+claim_str+"}"
	start_line_2 = "MERGE (entity_{}_{}_{}:{} {})".format(num, un_num, ent_num, categs, all_ent_props)
	final_line_2 = start_line_2+"\n"+relations_part


	if DEBUG:
		input ("final line 2 is {}".format(final_line_2))


	# ..add the results to our dict 
	# so as to not re-calculate something we know already
	# ents_already_met[ent[0]] = final_line
	ents_already_met[ent[0]] = final_line_2
	print ("And now we have {} ents".format(len(ents_already_met)))

	# return final_line
	return final_line_2



def sentence_cypher_line_generator(num, un_num, sent, var_name):
	# Let's make a special Label for sentences
	
	sent_name = "{"+"name:{}".format(type_prettifier(sent))+"}"

	start_line = "MERGE (sent_{}_{}:Sentence {})".format(num, un_num, sent_name)
	
	to_main_part = "MERGE (sent_{}_{})-[:RELATES_TO]->({})".format(num, un_num, var_name)

	sentence_cypher = " "+start_line+"\n"+to_main_part


	print ("sentence_cypher is {}".format(sentence_cypher))

	return sentence_cypher	



def openie_cypher_line_generator(openie, num, un_num):


	# ARGM-LOC, ARGM-PRD, 
	# ARGM-MNR (maybe "manner")
	# , ARGM-DIS, ARGM-ADV
	# R-ARG1, R-ARG0 and so on (these are usually "who")
	# ARGM-PRP ("purpose", supposedly). Probably relates to V


	# TODO: we lack quotes here
	# type_prettifier this stuff
	
	def filter_openie(openie_modified):
		
		v_props = {"temp_mod":"", 
					"purpose":"",
					"location":"",
					"adv_mod":""}

		if 'V' not in openie_modified:
			openie_modified['V'] = "DID_SOMETHING_TO"
			
		else:
			openie_modified['V'] = openie_modified['V'].upper()
			if "TOOK" in openie_modified['V']:
				if DEBUG:	
					input ("Ok, this V was TOOK. Is any of the ARGs 'place'?")
				if 'ARG2' in openie_modified:
					if "place" in openie_modified['ARG2']:
						openie_modified['V'] = openie_modified['V'].replace("TOOK", "TOOK_PLACE")
				if "part" in openie_modified['ARG1']:
					openie_modified['V'] = openie_modified['V'].replace("TOOK", "TOOK_PART")
		if 'ARG0' not in openie_modified:
		 	openie_modified['ARG0'] = "something"
		if 'ARG1' not in openie_modified:
		 	openie_modified['ARG1'] = "something"
		if 'ARG2' not in openie_modified:
		 	openie_modified['ARG2'] = "something" 
		
		
		if 'ARGM-TMP' in openie_modified:
			 # openie_modified['V'] += " {"+"temp_mod:{}".format(type_prettifier(openie_modified['ARGM-TMP']))+"}"
			#  input ("openie_modified['ARGM-TmP'] is {}".format(openie_modified['ARGM-TMP']))
			 v_props["temp_mod"] += type_prettifier(openie_modified['ARGM-TMP'], date=True)
			 del openie_modified['ARGM-TMP']		
		if 'ARGM-PRP' in openie_modified:
			 # openie_modified['V'] += " {"+"purpose:{}".format(type_prettifier(openie_modified['ARGM-PRP']))+"}"
			#  input ("openie_modified['ARGM-PRP'] is {}".format(openie_modified['ARGM-PRP']))
			 v_props["purpose"] += type_prettifier(openie_modified['ARGM-PRP'])
			 del openie_modified['ARGM-PRP']
		if 'R-ARG0' in  openie_modified:
			# admittedly, this might be a stupid thing to do...
			openie_modified['ARG0'] += " {"+"R-ARG0:\"who\""+"}"
			del openie_modified['R-ARG0']
		if 'R-ARG1' in  openie_modified:
			# admittedly, this might be a stupid thing to do...
			openie_modified['ARG1'] += " {"+"R-ARG1:\"who\""+"}"
			del openie_modified['R-ARG1']	
		if 'ARGM-LOC' in openie_modified:
			year_found = re.findall(r"(\d{3,4})", openie_modified['ARGM-LOC'])
			if year_found != []:
				print ("ARGM-LOC seems to contain a date instead of a location: {}".format(year_found))
				# openie_modified['V'] += " {"+"temp_mod:{}".format(type_prettifier(openie_modified['ARGM-LOC']))+"}"
				v_props["temp_mod"] += type_prettifier(openie_modified['ARGM-LOC'], date=True)
				del openie_modified['ARGM-LOC']		
			else:
				# input ("Perhaps our ARGM-LOC    {}    actually indicates a location".format(openie_modified['ARGM-LOC']))
				# input ("Note it")
				# openie_modified['V'] += " {"+"location:{}".format(type_prettifier(openie_modified['ARGM-LOC']))+"}"
				v_props["location"] += type_prettifier(openie_modified['ARGM-LOC'])

		if 'ARGM-ADV' in openie_modified:
			year_found = re.findall(r"(\d{3,4})", openie_modified['ARGM-ADV'])
			if year_found != []:
				print ("ARGM-ADV seems to contain a date instead of a location: {}".format(year_found))
				# openie_modified['V'] += " {"+"temp_mod:{}".format(type_prettifier(openie_modified['ARGM-ADV']))+"}"
				v_props["temp_mod"] += type_prettifier(openie_modified['ARGM-ADV'], date=True)

				del openie_modified['ARGM-ADV']		
			else:
				verbing = re.findall(r"((\w*?)ing)", openie_modified['ARGM-ADV'])
				if verbing != []:
					if DEBUG:	
						input ("Seems like we've found an interesting ARGM-ADV: {}".format(verbing))	
					# openie_modified['V'] += " {"+"adv_mod:{}".format(type_prettifier(openie_modified['ARGM-ADV']))+"}"
					v_props["adv_mod"] += type_prettifier(openie_modified['ARGM-ADV'])
		if 'ARGM-MNR' in openie_modified:
			year_found = re.findall(r"(\d{3,4})", openie_modified['ARGM-MNR'])
			if year_found != []:
				print ("ARGM-MNR seems to contain a date instead of a location: {}".format(year_found))		
				# openie_modified['V'] += " {"+"temp_mod:{}".format(type_prettifier(openie_modified['ARGM-MNR']))+"}"
				v_props["temp_mod"] += type_prettifier(openie_modified['ARGM-MNR'], date=True)
				del openie_modified['ARGM-MNR']		
		if 'ARGM-DIS' in openie_modified:
			year_found = re.findall(r"(\d{3,4})", openie_modified['ARGM-DIS'])
			if year_found != []:
				print ("ARGM-DIS seems to contain a date instead of a location: {}".format(year_found))
				# openie_modified['V'] += " {"+"temp_mod:{}".format(type_prettifier(openie_modified['ARGM-DIS']))+"}"
				v_props["temp_mod"] += type_prettifier(openie_modified['ARGM-DIS'], date=True)
				del openie_modified['ARGM-DIS']		
			elif openie_modified['ARGM-DIS'] == "also":
				if DEBUG:	
					input ("We've got an 'also' in our ARGM-DIS. It probably should be ignored or added to V")	

		# magically convert a dict into one string about properties 
		v_props_str = "{"+','.join(["{}:{}".format(it[0],it[1].replace("\"\"", ",")) for it in v_props.items() if it[1] != ""])+"}"	
		if DEBUG:
			input ("And now our v_props_str is {}".format(v_props_str))

		if v_props_str != "{"+"}":
			print ("Oh, something of substance!")
			openie_modified['V'] += " "+v_props_str

		print ("\tNow openie_modified is ")
		for thing in openie_modified.items():
			print ("{} : {}".format(thing[0], thing[1]))


		return openie_modified



	openie_modified = defaultdict(list)
	if openie == {' ':' '}:
		print ("And here we have an empty openie dict")
	openie_modified = openie

	print ("Initially, openie_modified is ")
	for thing in openie_modified.items():
		print ("{} : {}".format(thing[0], thing[1]))


	# Ok, we sometimes get arg1 and arg2, 
	# and sometimes arg0, arg1 and arg2 
	# and different relations
	# these cases probably need to produce different cypher lines
	# arg0-r->arg1, arg0-r->arg2
	# but sometimes it's just
	# arg1-r->arg2
	# there's also arg2-r->arg1
	# sometimes we even get an arg3


	
	if 'ARG1' in openie_modified and\
		'V' in openie_modified:
			if 'ARG0' in openie_modified:
				if 'ARG2' in openie_modified:

					openie_modified = filter_openie(openie_modified)

					arg0 = "{"+"name:{}".format(type_prettifier(openie_modified['ARG0']))+"}"
					arg1 = "{"+"name:{}".format(type_prettifier(openie_modified['ARG1']))+"}"
					arg2 = "{"+"name:{}".format(type_prettifier(openie_modified['ARG2']))+"}"
					v = openie_modified['V'].lstrip(" ")
					to_sent_part = "MERGE (arg0_{}_{})<-[:CONTAINS]-(sent_{}_{})".format(num, un_num, num, un_num)
					# input ("v is \n{}".format(v))

					print ("+++Here we might have pattern beta+++")
					merge_part = "MERGE (arg0_{}_{} {}) \nMERGE (arg1_{}_{} {})\nMERGE (arg2_{}_{} {})".format(num, un_num, arg0, num, un_num, arg1, num, un_num, arg2)
					
					relations_part = "MERGE (arg0_{}_{})-[:{}]->(arg1_{}_{}) \nMERGE (arg0_{}_{})-[:{}]->(arg1_{}_{})".format(num, un_num, v,  num, un_num,  num, un_num, v,  num, un_num)

					openie_cypher = merge_part+"\n"+relations_part+"\n"+to_sent_part
				
				else:

					openie_modified = filter_openie(openie_modified)

					arg0 = "{"+"name:{}".format(type_prettifier(openie_modified['ARG0']))+"}"
					arg1 = "{"+"name:{}".format(type_prettifier(openie_modified['ARG1']))+"}"
					v = openie_modified['V'].lstrip(" ")
					to_sent_part = "MERGE (arg0_{}_{})<-[:CONTAINS]-(sent_{}_{})".format(num, un_num, num, un_num)


					print ("+++Seems like we have pattern alpha+++")
					merge_part = "MERGE (arg0_{}_{} {})\nMERGE (arg1_{}_{} {})".format(num, un_num, arg0, num, un_num, arg1)
					relations_part = "MERGE (arg0_{}_{})-[:{}]->(arg1_{}_{})".format(num, un_num, v,  num, un_num)
			
					openie_cypher = merge_part+"\n"+relations_part+"\n"+to_sent_part

			elif 'ARG2' in openie_modified:

				openie_modified = filter_openie(openie_modified)

				arg1 = "{"+"name:{}".format(type_prettifier(openie_modified['ARG1']))+"}"
				arg2 = "{"+"name:{}".format(type_prettifier(openie_modified['ARG2']))+"}"
				v = openie_modified['V'].lstrip(" ")
				to_sent_part = "MERGE (arg1_{}_{})<-[:CONTAINS]-(sent_{}_{})".format(num, un_num, num, un_num)

				print ("+++Seems like we have pattern gamma+++")
				merge_part = "MERGE (arg2_{}_{} {})\nMERGE (arg1_{}_{} {})".format(num, un_num, arg2, num, un_num, arg1)
				relations_part = "MERGE (arg1_{}_{})-[:{}]->(arg2_{}_{})".format(num, un_num, v,  num, un_num)
			
				openie_cypher = merge_part+"\n"+relations_part+"\n"+to_sent_part

			elif 'ARG3' in openie_modified:
				print ("+++So, ARG1, V, ARG3. That's also rare+++")

				openie_modified = filter_openie(openie_modified)

				arg1 = "{"+"name:{}".format(type_prettifier(openie_modified['ARG1']))+"}"
				arg3 = "{"+"name:{}".format(type_prettifier(openie_modified['ARG3']))+"}"
				v = openie_modified['V'].lstrip(" ")
				to_sent_part = "MERGE (arg1_{}_{})<-[:CONTAINS]-(sent_{}_{})".format(num, un_num, num, un_num)

				merge_part = "MERGE (arg3_{}_{} {})\nMERGE (arg1_{}_{} {})".format(num, un_num, arg3, num, un_num, arg1)
				relations_part = "MERGE (arg1_{}_{})-[:{}]->(arg3_{}_{})".format(num, un_num, v,  num, un_num)

				openie_cypher = merge_part+"\n"+relations_part+"\n"+to_sent_part

			else:
				print ("+++Just ARG1 and V. That's not a lot+++")
				
				openie_modified = filter_openie(openie_modified)

				
				arg0 = "{"+"name:{}".format(type_prettifier( type_prettifier("something")))+"}"
				arg1 = "{"+"name:{}".format(type_prettifier(openie_modified['ARG1']))+"}"
				v = openie_modified['V'].lstrip(" ")
				to_sent_part = "MERGE (arg0_{}_{})<-[:CONTAINS]-(sent_{}_{})".format(num, un_num, num, un_num)


				merge_part = "MERGE (arg0_{}_{} {})\nMERGE (arg1_{}_{} {})".format(num, un_num, arg0, num, un_num, arg1)
				relations_part = "MERGE (arg0_{}_{})-[:{}]->(arg1_{}_{})".format(num, un_num, v,  num, un_num)
		
				openie_cypher = merge_part+"\n"+relations_part+"\n"+to_sent_part

	else:
		print ("Probably an empty openie_modified")
		openie_modified = filter_openie(openie_modified)

		arg0 = "{"+"name:{}".format(type_prettifier(openie_modified['ARG0']))+"}"
		arg1 = "{"+"name:{}".format(type_prettifier(openie_modified['ARG1']))+"}"
		v = openie_modified['V'].lstrip(" ")
		to_sent_part = "MERGE (arg0_{}_{})<-[:CONTAINS]-(sent_{}_{})".format(num, un_num, num, un_num)


		merge_part = "MERGE (arg0_{}_{} {})\nMERGE (arg1_{}_{} {})".format(num, un_num, arg0, num, un_num, arg1)
		relations_part = "MERGE (arg0_{}_{})-[:{}]->(arg1_{}_{})".format(num, un_num, v,  num, un_num)

		openie_cypher = merge_part+"\n"+relations_part+"\n"+to_sent_part


	# print ("openie_modified['V'] = {}".format(openie_modified['V']))
	
	

	print ("openie_cypher is {}".format(openie_cypher))
	# input ("Once you've inspected the line and the dict to your satisfaction, press any key...")

	return openie_cypher




def chunk_cypher_line_generator(chunk, var_name):
	ch = chunk

	chunk_cypher = []
	
	
	clusters = ch[1][0]['clusters']
	chunk = ch[1][0]['chunk']
	num = ch[0]

	
	# input ("clusters are {}".format(clusters))
	
	cluster_ents = defaultdict(list)
	cluster_ents_lines = []

	

	for clust_key in clusters.keys():
		# print ("clusters[{}] is {}".format(clust_key, clusters[clust_key]))
		# for point in clusters[clust_key]:
		# 	cluster_ents.append()

		# some python magic: find the longest string in a list!
		if clusters[clust_key] == []:
			print ("Oh no, clust_key is {}, but clusters[clust_key] is {}".format(clust_key, clusters[clust_key]))
		else:
			cluster_ent_name = max(clusters[clust_key], key=len)

			cluster_ents[clust_key] = cluster_ent_name
			# append("CREATE cluster_ent_{} SET cluster_ent_{}.name={}".format(clust_key, clust_key, ))
			

			
			# cluster_ents_lines.append("""CREATE ({}_cluster_ent_{}) SET {}_cluster_ent_{}.name = {} """.format(num, clust_key, num, clust_key, type_prettifier(cluster_ent_name) ))
			cluster_name = "{"+"name:{}".format(type_prettifier(cluster_ent_name))+"}"
			cluster_ents_lines.append("""CREATE ({}_cluster_ent_{} {}) """.format(num, clust_key, cluster_name))
	# input ("And now cluster_ents is {}".format(cluster_ents))	

	# for in cluster_ents.items():
	# 	cluster


	# print ("{} is {}".format(num,chunk))
	print ("num is {}".format(num))
	for un_num,unit in enumerate(chunk):
		unit_modified = defaultdict(list)
		sent = unit['sentence']
		if num == "chunk_0" and un_num == 0:
			first_sentence_parser(sent)
		unit_modified['sentence'] = sent

		print ("clusters are {}".format(clusters))

		if not clusters == {}:
			for clust_key in clusters.keys():
				found_cluster_lines = []
				for point in clusters[clust_key]:
					# unless we limit the search, 
					# strings like "he"
					# will be found in The and There
					if point in sent and len(point) > 3:
						# input ("Great news! We've found {} in {}".format(point, sent)) #\n We probably should make a cypher line about it
						cluster_ent_line = "\nMERGE (sent_{}_{})-[:REFERENCES]->(cluster_ent_{})".format(num, un_num, clust_key)
						found_cluster_lines.append(cluster_ent_line)

			found_cluster_one_line = '\n'.join(found_cluster_lines)
		else:
			found_cluster_one_line = ""			

		sentence_cypher = sentence_cypher_line_generator(num, un_num, sent, var_name)

		sentence_cypher += " "+found_cluster_one_line

		

		chunk_cypher.append(sentence_cypher)
		openie = json.loads(unit['openie'])
		# print ("openie has type {}".format(type(openie)))
		# openie = unit['openie']
		ents = unit['entities']
		# print ("ents is {} of type {}".format(ents, type(ents)))
		print ("=======================================")
		# print ("\t sentence is {}".format(sent))
		print ("\t sentence is {}".format(unit_modified['sentence']))
		print ("\t openie:")
		print ("\topenie = {}".format(openie))

		openie_cypher = openie_cypher_line_generator(openie, num, un_num)


		
		chunk_cypher.append(openie_cypher)

		
		print ("\t entities:")
		if not ents == {}:
			ents_modified = defaultdict(list)
			ent_cypher_list = []

			for ent_num, ent in enumerate(ents.items()):
				categs = []
				categ_parses = re.findall(r"((.*?) (\((0\.\d{1,4})\)|\(1\.0\)))",ent[1])
				# print ("categ_parses is {}".format(categ_parses))
				confidence = categ_parses[0][3]
				
				categ = categ_parses[0][1]
				categ = categ[0]+categ[1:].lower()  #PERSON -> Person and so on
				categs.append(categ) 


				
				print ("\t\t{}:{}  (confidence is {})".format(ent[0], ent[1], confidence))
				# input ("Try searching for it yourself..")
				
				if categ not in ["Date", "Cardinal", "Ordinal"]:
					# let's try the wiki
					site = pywikibot.Site("en", "wikipedia")
					# alright, we need an actual name for this
					# not just initials

					try:
						if "]" in ent[0] or "[" in ent[0]:
							ent_copy = ent[0].replace("[","").replace("]","")
						else:
							ent_copy = ent[0]
						page = pywikibot.Page(site, "{}".format(ent_copy))
						item = pywikibot.ItemPage.fromPage(page)
						item_dict = item.get()

						print("\t\t\tFor entity {} we found {} in the wiki".format(ent[0],item))
						# print ("item has type {}".format(type(item)))
						item_wikidata_num = re.findall(r"\[\[(wikidata:(\w(\d*)))\]\]", str(item))[0][1]
						print ("\t\t\titem_wikidata_num is {}".format(item_wikidata_num))
						# print ("item to string:{}".format(str(item)))
						# print ("item_dict is {}".format(item_dict))
						# print ("\t\t\t item_dict. keys are {}".format(item_dict.keys()))
						

						item_stuff = defaultdict(list)
						for key in item_dict.keys():
							thingy = item_dict["{}".format(key)]
							# print ("\t\t\t\titem_dict[\"{}\"] = ".format(key))
							if key == "labels":
								# print ("Yoohoo! Labels found!")
								for item in thingy.items():
									if item[0] =="en":
										# item_label_en = item[1]
										item_stuff['item_label_en'] = item[1] 
										# print ("now item_stuff['item_label_en'] = {}".format(item_stuff['item_label_en']))
									elif item[0] =="ru":
										# item_label_ru = item[1]
										item_stuff['item_label_ru'] = item[1]
								# print ("and now item_stuff is {}".format(item_stuff))		
							elif key == "descriptions":
								for item in thingy.items():
									if item[0] =="en":
										# item_desc_en = item[1]
										item_stuff['item_desc_en'] = item[1]
									elif item[0] =="ru":
										# item_desc_ru = item[1]
										item_stuff['item_desc_ru'] = item[1]
							elif key == "aliases":
								for item in thingy.items():
									if item[0] =="ru":
										# item_alias_ru = item[1]
										item_stuff['item_alias_ru'] = item[1]
							
							elif key == "claims":
								print ("Preparing to deal with it...")
								item_claims = get_statements(item_wikidata_num)
								# input(" Go on...")
					
						wikidata_categ = ""
						# input ("Before processing, wikidata_categ is {}".format(wikidata_categ))
						if 'instance of' in item_claims:
							print ("\t\t\t item_claims['instance of'] is {}".format(item_claims['instance of']))
							wikidata_categ = item_claims['instance of']
							wikidata_categ = wikidata_categ[0].upper()+prettify(wikidata_categ[1:])
							# wikidata_categ = prettify(wikidata_categ)
							print ("\t\t\t wikidata_categ is {}".format(wikidata_categ))
						if 'subclass of' in item_claims:
							wikidata_categ = item_claims['subclass of']
							# input ("Before processing, wikidata_categ is {}".format(wikidata_categ))
							wikidata_categ = wikidata_categ[0].upper()+prettify(wikidata_categ[1:])
							# wikidata_categ = prettify(wikidata_categ)
							print ("\t\t\t wikidata_categ is {}".format(wikidata_categ))	
						categs.append(wikidata_categ)
						

						categs = ':'.join(categs)

						ent_cypher = ent_cypher_line_generator(ent, num, un_num, ent_num, categs, confidence, item_stuff=item_stuff, item_claims=item_claims)
						if ent_cypher is not None:
							ent_cypher_list.append(ent_cypher)
						# input ("Cool cypher string, eh?")

					except pywikibot.exceptions.NoPage:
						print ("\t\t\tNope, no banana! Sorry ;-/")

						categs = ':'.join(categs)

						ent_cypher = ent_cypher_line_generator(ent, num, un_num, ent_num, categs, confidence)
						if ent_cypher is not None:
							ent_cypher_list.append(ent_cypher)

			ent_cypher_string = '\n'.join(ent_cypher_list)	
			chunk_cypher.append(ent_cypher_string)			 
				# print ("\t\t ent_cypher is {}".format(ent_cypher))			 
				
		# print ("\t entities is {} of type {}".format(ents, type(ents)))
	# print ("{} \n of type {}".format(aug_sents, type(aug_sents)))  
	# print ("{}".format(thingy))   

	for _,chunk_cypher_line in enumerate(chunk_cypher):
		print ("chunk line {} is {}".format(_,chunk_cypher_line))


	cluster_ents_one_line = '\n'.join(cluster_ents_lines)	

	chunk_cypher_string = '\n'.join(chunk_cypher)  

	chunk_cypher_string = cluster_ents_one_line+"\n"+chunk_cypher_string
	
	
	
	print ("chunk_cypher_string is {}".format(chunk_cypher_string))



	return chunk_cypher_string




def to_graph(our_object):
	all_chunk_cyphers = []
	cypher_lines = []

	if DEBUG:	
		input ("All right, to_graph has received object {}".format(our_object))

	thingy = json.loads(json.dumps(our_object, indent=2))
	meta = thingy['meta'][2:-3]
	
	props = thingy['props']
	props_parser(props)
	

	url = thingy['url']
	print ("{}".format(url))
	
	house_num = re.findall(r"http://www.citywalls.ru/house(\d{1,5})+.html", url)[0]
	# print ("house_num = {} of type {}".format(house_num, type(house_num)))
	# print ("house_num = {}".format(house_num))
	input ("house_num = {}".format(house_num))
	
	var_name = "citywalls_{}".format(house_num)
	

	saved_matches = thingy['saved_matches']
	assoc_photos = thingy['assoc_photos']
	json_part = thingy['json_part']
	aug_sents = json_part['aug_sents']
	
	meta_part_parser(meta)	
	
	#print ("{}".format(props))
	
	sav_match_parser(saved_matches)

	# print ("{}".format(assoc_photos))

	for ch in aug_sents.items():
		chunk_cypher = chunk_cypher_line_generator(ch, var_name)
		# architects, constr_year, arch_style = chunk_cypher_line_generator(ch, var_name, props)[1:]
		all_chunk_cyphers.append(chunk_cypher)


	all_chunk_cyphers_str = '\n'.join(all_chunk_cyphers)	

	

	# TODO: fix it
	Category = "SomeCategory" 
	name = for_initial['name']


	if DEBUG:
		input ("Ok, we should be calling cypher_initial_former now...")
	cypher_initial = cypher_initial_former(var_name, name, Category, url, assoc_photos)
	


	all_chunk_cyphers_str = cypher_initial+ "\n " + all_chunk_cyphers_str 				

	if DEBUG:	
		input ("all_chunk_cyphers_str is {}".format(all_chunk_cyphers_str))




	for chunk_cypher in all_chunk_cyphers:
		print ("/////////////////////////////////////")
		print (chunk_cypher)
	print ("++++++++++++++++++++++++++++++++++++++")    


	
	return all_chunk_cyphers_str	 




objects = []

with (open("/content/drive/My Drive/citywalls/improved_all.pkl", "rb")) as openfile:
    while True:
        try:
            objects.append(pickle.load(openfile))
        except EOFError:
            break


input ("Dude, have you implemented all the TODOs?..")
# Admin's house is around 448
for num, our_object in enumerate(tqdm(objects[810:])):
	print ("Processing object #{}".format(num))
	complete_cypher_code = to_graph(our_object)
	with open("/content/drive/My Drive/citywalls/citywalls_all.cypher", "a+") as f:
		f.write(complete_cypher_code)
		f.write("\n++++++++++++++++++++++++++++++++++++++++++++\n")

Dude, have you implemented all the TODOs?..





  0%|          | 0/46186 [00:00<?, ?it/s]

Processing object #0
We've just got props {'Архитекторы:': 'Митурич Н. А. Демков Н. Ф. Крылов В. К.', 'Год постройки:': '1928-1929', 'Стиль:': 'Ретроспективизм'}
Архитекторы:: Митурич Н. А. Демков Н. Ф. Крылов В. К.
architects is now [('Митурич', 'Н', 'А'), ('Демков', 'Н', 'Ф'), ('Крылов', 'В', 'К')] of type <class 'list'>
architects[0] = ('Митурич', 'Н', 'А') of type <class 'tuple'>
this tuple has element Митурич
this tuple has element Н
this tuple has element А
architects_modified is now ['Митурич Н. А.', 'Демков Н. Ф.', 'Крылов В. К.']
arch is ('Митурич', 'Н', 'А')
arch is ('Демков', 'Н', 'Ф')
arch is ('Крылов', 'В', 'К')
Год постройки:: 1928-1929
constr_year is 1928-1929
Стиль:: Ретроспективизм
arch_style is Ретроспективизм
http://www.citywalls.ru/house12235.html
house_num = 12235
meta is ['Фотографии Клуб им. И.И. Орлова - Жилой дом, Нежинская ул., 8, архитектурный стиль Ретроспективизм, архитектор Митурич Н. А., Демков Н. Ф., Крылов В. К.', 'санкт-петербург, Клуб им. И.И. Орлова 




  0%|          | 1/46186 [00:27<350:49:39, 27.35s/it]

/////////////////////////////////////
CREATE (chunk_0_cluster_ent_3 {name:"The I.I. Orlov Club - 8 Nezhinskaya St. Residential House"}) 
CREATE (chunk_0_cluster_ent_0 {name:"Count Pavel Alexandrovich Bennigsen , who with his wife Aleksandra Karlovna lives at 32 Kirochna Street"}) 
CREATE (chunk_0_cluster_ent_1 {name:"Yakov Nikolaevich 's son , Nikolai Yakovlevich Molvo ( September 17 , 1809 - September 25 , 1870 )"}) 
CREATE (chunk_0_cluster_ent_2 {name:"the Department of Consumer Services of the Leningrad City Executive Committee"}) 
CREATE (chunk_0_cluster_ent_4 {name:"Main Geophysical Observatory named after A.I. Voeykov"}) 
CREATE (chunk_0_cluster_ent_5 {name:"the fencing of the Summer Garden Palace embankment"}) 
CREATE (chunk_0_cluster_ent_6 {name:"Sofia Bogdanovna Tomilova , his wife"}) 
CREATE (chunk_0_cluster_ent_8 {name:"Leningrad Soft Furniture Factory No . 1"}) 
CREATE (chunk_0_cluster_ent_9 {name:"a stand - alone 105-meter tower of a business center"}) 
CREATE (chunk_0_clu

...




  0%|          | 9/46186 [05:06<537:52:31, 41.93s/it]

/////////////////////////////////////
CREATE (chunk_0_cluster_ent_0 {name:"Count Pavel Alexandrovich Bennigsen , who with his wife Aleksandra Karlovna lives at 32 Kirochna Street"}) 
CREATE (chunk_0_cluster_ent_1 {name:"Yakov Nikolaevich 's son , Nikolai Yakovlevich Molvo ( September 17 , 1809 - September 25 , 1870 )"}) 
CREATE (chunk_0_cluster_ent_3 {name:"a dry ice plant built near the Badaevskaya station"}) 
CREATE (chunk_0_cluster_ent_2 {name:"the Department of Consumer Services of the Leningrad City Executive Committee"}) 
CREATE (chunk_0_cluster_ent_4 {name:"Main Geophysical Observatory named after A.I. Voeykov"}) 
CREATE (chunk_0_cluster_ent_5 {name:"the fencing of the Summer Garden Palace embankment"}) 
CREATE (chunk_0_cluster_ent_6 {name:"Sofia Bogdanovna Tomilova , his wife"}) 
CREATE (chunk_0_cluster_ent_8 {name:"Leningrad Soft Furniture Factory No . 1"}) 
CREATE (chunk_0_cluster_ent_9 {name:"a stand - alone 105-meter tower of a business center"}) 
CREATE (chunk_0_cluster_en




  0%|          | 10/46186 [05:43<522:14:02, 40.71s/it]

/////////////////////////////////////
CREATE (chunk_0_cluster_ent_0 {name:"Count Pavel Alexandrovich Bennigsen , who with his wife Aleksandra Karlovna lives at 32 Kirochna Street"}) 
CREATE (chunk_0_cluster_ent_1 {name:"Yakov Nikolaevich 's son , Nikolai Yakovlevich Molvo ( September 17 , 1809 - September 25 , 1870 )"}) 
CREATE (chunk_0_cluster_ent_3 {name:"a dry ice plant built near the Badaevskaya station"}) 
CREATE (chunk_0_cluster_ent_2 {name:"the Department of Consumer Services of the Leningrad City Executive Committee"}) 
CREATE (chunk_0_cluster_ent_4 {name:"Main Geophysical Observatory named after A.I. Voeykov"}) 
CREATE (chunk_0_cluster_ent_5 {name:"the fencing of the Summer Garden Palace embankment"}) 
CREATE (chunk_0_cluster_ent_6 {name:"Sofia Bogdanovna Tomilova , his wife"}) 
CREATE (chunk_0_cluster_ent_8 {name:"Leningrad Soft Furniture Factory No . 1"}) 
CREATE (chunk_0_cluster_ent_9 {name:"a stand - alone 105-meter tower of a business center"}) 
CREATE (chunk_0_cluster_en

KeyboardInterrupt: ignored